In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from pathlib import Path
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_columns', None)


import matplotlib.pyplot as plt
import seaborn as sns

# Load data

In [2]:
train_X = pd.read_csv('../data/train_values.csv', index_col='building_id')
train_y = pd.read_csv('../data/train_labels.csv', index_col='building_id')

X_test = pd.read_csv('../data/test_values.csv', index_col='building_id')

# Explore features

In [3]:
train_X[['geo_level_1_id','geo_level_2_id','geo_level_3_id']] = train_X[['geo_level_1_id','geo_level_2_id','geo_level_3_id']].astype(str)
X_test[['geo_level_1_id','geo_level_2_id','geo_level_3_id']] = X_test[['geo_level_1_id','geo_level_2_id','geo_level_3_id']].astype(str)

train_X[['count_floors_pre_eq','age','area_percentage','height_percentage']] = train_X[['count_floors_pre_eq','age','area_percentage','height_percentage']].astype(float)

train_X['volume_percentage']=train_X['area_percentage'] * train_X['height_percentage']
X_test['volume_percentage']=X_test['area_percentage'] * X_test['height_percentage']

# Categorical columns 
categorical_columns = [c for c in train_X.select_dtypes(include=['object'])]
numerical_columns= list(set(train_X.columns) - set(categorical_columns))

In [4]:
# Drop building_id (index) from X and y
train_X.reset_index(drop=True, inplace=True)
train_y.reset_index(drop=True, inplace=True)

In [5]:
duplicate_index_mask = train_X.index.duplicated(keep='first')
X = train_X[~duplicate_index_mask]
y = train_y[~duplicate_index_mask]

# Splitting the data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
from sklearn.utils import resample

def upsample(X_train, y_train):
    data = pd.concat([X_train, y_train], axis=1)

    # Separate classes
    damage_3 = data[data['damage_grade']==3]
    damage_2 = data[data['damage_grade']==2]
    damage_1  = data[data['damage_grade']==1]
 
    # Upsample miniroty class, damage=1
    damage_1_upsampled = resample(damage_1, 
                                  replace=True,     # sample with replacement
                                  n_samples=damage_2.shape[0]) # reproducible results
    
    # Upsample miniroty class, damage=3
    damage_3_upsampled = resample(damage_3, 
                                  replace=True,     # sample with replacement
                                  n_samples=damage_2.shape[0]) # reproducible results

    # Combine majority class with upsampled minority classes
    data_upsampled = pd.concat([damage_1_upsampled, damage_2, damage_3_upsampled])

    y_train_upsampled = data_upsampled[['damage_grade']]
    X_train_upsampled = data_upsampled.drop(['damage_grade'], axis=1)

    return (X_train_upsampled, y_train_upsampled)

X_train_new, y_train_new = upsample(X_train=X_train, y_train=y_train)

## $\chi^2$ Analysis

In [7]:
from scipy.stats import chi2_contingency

In [8]:
chi2_reults = []

for feat in categorical_columns:
    contingency_table = pd.crosstab(X_train[feat],y_train.squeeze())
    chi2,   p, dof, expected = chi2_contingency(contingency_table)
    chi2_reults.append((feat, chi2, p))

chi2_square_results_df = pd.DataFrame(chi2_reults, columns=["Feature", "Chi-square", "P-value"])

In [9]:
chi2_square_results_df

,Feature,Chi-square,P-value
0,geo_level_1_id,58148.229423,0.000000e+00
1,geo_level_2_id,107939.611770,0.000000e+00
2,geo_level_3_id,149625.719464,0.000000e+00
3,land_surface_condition,358.589261,2.450835e-76
4,foundation_type,38989.276303,0.000000e+00
5,roof_type,24275.220527,0.000000e+00
6,ground_floor_type,29176.796855,0.000000e+00
7,other_floor_type,25379.130262,0.000000e+00
8,position,851.454353,1.170333e-180
9,plan_configuration,1460.078797,1.794511e-299


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming chi2_square_results_df is your DataFrame
chi2_square_results_df = pd.DataFrame(chi2_reults, columns=["Feature", "Chi-square", "P-value"])

# Create subplots: one for Chi-square values and one for P-values
fig = make_subplots(rows=2, cols=1, subplot_titles=("Chi-square values", "P-values"))

# Bar plot for Chi-square values
fig.add_trace(
    go.Bar(x=chi2_square_results_df["Feature"], y=chi2_square_results_df["Chi-square"], name="Chi-square"),
    row=1, col=1
)

# Scatter plot for P-values
fig.add_trace(
    go.Scatter(x=chi2_square_results_df["Feature"], y=chi2_square_results_df["P-value"], mode="markers", name="P-value"),
    row=2, col=1
)

# Update layout
fig.update_layout(height=600, width=800, title_text="Chi-square Analysis Results")
fig.update_yaxes(title_text="Chi-square Value", row=1, col=1)
fig.update_yaxes(title_text="P-value", row=2, col=1)
fig.update_xaxes(title_text="Features", row=2, col=1)

# Show plot
fig.show()


## Encode categorical values

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import TargetEncoder


# Preprocessing categorical data
categorical_transformer = Pipeline(steps=[
    ("target", TargetEncoder(target_type="continuous"))
])

# Bundle prepocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_columns),
        ("numerical", "passthrough", numerical_columns),
    ])

# Model Training

In [14]:
y_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,volume_percentage
5654,20,281,7097,2,15,6,7,t,r,q,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,42
28094,26,886,12157,2,10,6,6,t,w,n,f,q,s,d,0,0,0,0,0,0,1,0,0,0,0,v,0,0,0,0,0,0,0,0,0,0,0,0,36
151910,26,36,1125,1,0,5,3,n,r,n,v,j,s,d,0,0,0,0,0,1,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,15
53449,13,1372,4555,2,10,6,5,t,r,n,f,q,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,30
202567,8,206,6064,3,35,6,5,t,r,q,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,30


In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=57)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
]
)

clf.fit(X_train, y_train["damage_grade"])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('target',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'land_surface_condition',
                                                   'foundation_type',
                                                   'roof_type',
                                                   'ground_floor_type',
                                                   'other_floor_type',
                                                   'position',
                                                   'plan_configuration',
                                                   'legal_ownership_status']),
                                                 ('num...
                                                   'count_families',
                                                   'has_superstructure_mud_mortar_stone',
                                                   'has_superstructure_rc_non_engineered',
                                                   'has_secondary_use_rental',
                                                   'has_superstructure_cement_mortar_brick',
                                                   'has_secondary_use_other',
                                                   'age',
                                                   'has_superstructure_cement_mortar_stone',
                                                   'has_secondary_use',
                                                   'has_secondary_use_hotel'])])),
                ('model',
                 RandomForestClassifier(max_depth=50, random_state=57))])

In [16]:
import plotly.graph_objects as go

# Extract feature importances from the random forest model
feature_importances = clf.named_steps['model'].feature_importances_

# Combine feature importances with their corresponding feature names
# This requires a bit of manipulation since you have a ColumnTransformer
# Assuming 'categorical_columns' and 'numerical_columns' are lists of feature names
features = categorical_columns + numerical_columns
sorted_idx = feature_importances.argsort()

# Create a bar plot
fig = go.Figure([go.Bar(x=feature_importances[sorted_idx], y=[features[i] for i in sorted_idx], orientation='h')])

# Update layout
fig.update_layout(title='Feature Importances in Random Forest Model',
                  xaxis_title='Importance',
                  yaxis_title='Feature',
                  yaxis={'categoryorder':'total ascending'},
                  height=600, width=800)

# Show the plot
fig.show()


In [12]:
from sklearn.metrics import f1_score

pred_valid = clf.predict(X_valid)

my_f1_score = f1_score(y_valid, pred_valid, average='micro')

print(f"F1 score: {my_f1_score}")

F1 score: 0.7413134820897527


In [13]:
preds = clf.predict(X_test)

my_submission = pd.DataFrame(data=preds,
                             columns=['damage_grade'],
                             index=X_test.index)

my_submission.to_csv('submission.csv')

In [6]:
y_train_encoded = y_train.copy() - 1
y_train_encoded

,damage_grade
building_id,
802906,2
28830,1
94947,2
590882,1
201944,2
...,...
688636,1
669485,2
602512,2


In [15]:
y_valid_encoded = y_valid.copy() - 1
y_valid_encoded 

,damage_grade
building_id,
315409,1
838451,2
808750,1
322968,1
212418,2
...,...
62808,2
756596,1
574520,2


## XGBoost

In [7]:
from xgboost import XGBClassifier

xg_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=57,
#    use_label_encoder=False, # Use this to avoid a deprecation warning from XGBoost
    objective='multi:softmax', # Specify the multi-class objective
    num_class=3 # Number of classes in the target variable
)

# Create the pipeline with XGBoost
xg_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", xg_model)
])

# Fitting the model
# xg_clf.fit(X_train, y_train_encoded["damage_grade"])
xg_clf



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('target',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'land_surface_condition',
                                                   'foundation_type',
                                                   'roof_type',
                                                   'ground_floor_type',
                                                   'other_floor_type',
                                                   'position',
                                                   'plan_configuration',
                                                   'legal_ownership_status']),
                                                 ('num...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_class=3, num_parallel_tree=None,
                               objective='multi:softmax', ...))])

In [17]:
xg_preds = xg_clf.predict(X_valid)

In [18]:
xg_f1_score = f1_score(y_valid_encoded, xg_preds, average='micro')

print(f"F1 score: {xg_f1_score}")

F1 score: 0.7372460236756777


## Grid Search

In [8]:
from sklearn.model_selection import GridSearchCV

In [17]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.05, 0.1],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=xg_clf, param_grid=param_grid, 
                           scoring='f1_micro', n_jobs=-1, cv=3, verbose=3)


In [18]:
# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train_encoded["damage_grade"])

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[CV 1/3] END model__learning_rate=0.01, model__n_estimators=100;, score=0.728 total time= 5.1min
[CV 3/3] END model__learning_rate=0.01, model__n_estimators=100;, score=0.728 total time= 5.1min
[CV 2/3] END model__learning_rate=0.01, model__n_estimators=100;, score=0.729 total time= 5.2min
[CV 1/3] END model__learning_rate=0.01, model__n_estimators=200;, score=0.729 total time= 9.9min
[CV 2/3] END model__learning_rate=0.01, model__n_estimators=200;, score=0.731 total time=10.0min
[CV 3/3] END model__learning_rate=0.01, model__n_estimators=200;, score=0.730 total time=10.0min
[CV 1/3] END model__learning_rate=0.05, model__n_estimators=100;, score=0.733 total time= 5.0min
[CV 2/3] END model__learning_rate=0.05, model__n_estimators=100;, score=0.735 total time= 5.1min
[CV 3/3] END model__learning_rate=0.05, model__n_estimators=100;, score=0.734 total time= 5.4min
[CV 1/3] END model__learning_rate=0.01, model__n_estimators=300;, score=0.731 total time=15.5min
[CV 2/3] END model__learning_r

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('target',
                                                                                          TargetEncoder(target_type='continuous'))]),
                                                                         ['geo_level_1_id',
                                                                          'geo_level_2_id',
                                                                          'geo_level_3_id',
                                                                          'land_surface_condition',
                                                                          'foundation_type',
                                                                          'roof_type',
                                                                          'ground_floor_type',
                                                                          'other_floor_type',
                                                                          'position',
                                                                          'plan_configuration',
                                                                          'leg...
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=5,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None, num_class=3,
                                                      num_parallel_tree=None,
                                                      objective='multi:softmax', ...))]),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.01, 0.05, 0.1],
                         'model__n_estimators': [100, 200, 300]},
             scoring='f1_micro', verbose=3)

In [19]:
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)

Best Parameters: {'model__learning_rate': 0.1, 'model__n_estimators': 300}


In [20]:
best_score = grid_search.best_score_
print("Best Score:", best_score)

Best Score: 0.7386502737902004


In [21]:
best_estimator = grid_search.best_estimator_
print("Best Estimator:", best_estimator)

Best Estimator: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('target',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'land_surface_condition',
                                                   'foundation_type',
                                                   'roof_type',
                                                   'ground_floor_type',
                                                   'other_floor_type',
                                                   'position',
                                                   'plan_configuration',
         

In [29]:
best_xg_preds = best_estimator.predict(X_test) +1

best_xg_submission = pd.DataFrame(data=best_xg_preds,
                             columns=['damage_grade'],
                             index=X_test.index)

best_xg_submission.to_csv('best_xg_submission.csv')

In [27]:
best_xg_submission_df.to_csv('best_xg_submission.csv')

In [29]:
xg_model = XGBClassifier(
    n_estimators=11,
    learning_rate=0.1,
    max_depth=5,
    random_state=57,
    use_label_encoder=False, # Use this to avoid a deprecation warning from XGBoost
    objective='multi:softmax', # Specify the multi-class objective
    num_class=3 # Number of classes in the target variable
)

# Create the pipeline with XGBoost
xg_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", xg_model)
])

# Fitting the model
xg_clf.fit(X_train, y_train_encoded)


/home/safak/.local/lib/python3.10/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('target',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'land_surface_condition',
                                                   'foundation_type',
                                                   'roof_type',
                                                   'ground_floor_type',
                                                   'other_floor_type',
                                                   'position',
                                                   'plan_configuration',
                                                   'legal_ownership_status']),
                                                 ('num...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.1,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=5, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=11, n_jobs=0, num_class=3,
                               num_parallel_tree=1, objective='multi:softmax', ...))])

In [30]:
xg_preds = xg_clf.predict(X_valid)

## Reducing the data to binary

In [13]:
def scaler(n):
    if n == 1:
        return 0
    if n>1:
        return 1

In [18]:
y["binary_damage"] = y["damage_grade"].apply(scaler)

In [20]:
new_y = y.drop(columns="damage_grade")

In [21]:
new_y

,binary_damage
0,1
1,1
2,1
3,1
4,1
...,...
260596,1
260597,1
260598,1
260599,1


In [22]:
binary_X_train, binary_X_valid, binary_y_train, binary_y_valid = train_test_split(X, new_y, test_size=0.2, random_state=0)

In [23]:
binary_y_train.head()

,binary_damage
5654,0
28094,0
151910,0
53449,1
202567,1


In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import TargetEncoder


# Preprocessing categorical data
categorical_transformer = Pipeline(steps=[
    ("target", TargetEncoder(target_type="continuous"))
])

# Bundle prepocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_columns),
        ("numerical", "passthrough", numerical_columns),
    ])

In [30]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=57)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
]
)

clf.fit(binary_X_train, binary_y_train["binary_damage"])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('target',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'land_surface_condition',
                                                   'foundation_type',
                                                   'roof_type',
                                                   'ground_floor_type',
                                                   'other_floor_type',
                                                   'position',
                                                   'plan_configuration',
                                                   'legal_ownership_status']),
                                                 ('num...
                                                   'has_superstructure_mud_mortar_stone',
                                                   'has_superstructure_rc_non_engineered',
                                                   'has_secondary_use_rental',
                                                   'has_superstructure_cement_mortar_brick',
                                                   'has_secondary_use_other',
                                                   'age',
                                                   'has_superstructure_cement_mortar_stone',
                                                   'has_secondary_use',
                                                   'has_secondary_use_hotel'])])),
                ('model',
                 RandomForestClassifier(max_depth=100, n_estimators=300,
                                        random_state=57))])

In [31]:
from sklearn.metrics import f1_score

binary_pred_valid = clf.predict(binary_X_valid)

binary_f1_score = f1_score(binary_y_valid, binary_pred_valid, average='micro')

print(f"F1 score: {binary_f1_score}")

F1 score: 0.9302776232228852


In [32]:
import plotly.graph_objects as go

# Extract feature importances from the random forest model
feature_importances = clf.named_steps['model'].feature_importances_

# Combine feature importances with their corresponding feature names
# This requires a bit of manipulation since you have a ColumnTransformer
# Assuming 'categorical_columns' and 'numerical_columns' are lists of feature names
features = categorical_columns + numerical_columns
sorted_idx = feature_importances.argsort()

# Create a bar plot
fig = go.Figure([go.Bar(x=feature_importances[sorted_idx], y=[features[i] for i in sorted_idx], orientation='h')])

# Update layout
fig.update_layout(title='Feature Importances in Random Forest Model',
                  xaxis_title='Importance',
                  yaxis_title='Feature',
                  yaxis={'categoryorder':'total ascending'},
                  height=600, width=800)

# Show the plot
fig.show()
